In [ ]:
#darger

In [4]:
pip install pyghelpers


     ---------------------------------------- 0.0/802.3 kB ? eta -:--:--
      --------------------------------------- 10.2/802.3 kB ? eta -:--:--
      ------------------------------------ 20.5/802.3 kB 162.5 kB/s eta 0:00:05
     - ----------------------------------- 30.7/802.3 kB 186.2 kB/s eta 0:00:05
     -- ---------------------------------- 61.4/802.3 kB 328.2 kB/s eta 0:00:03
     ------------- ------------------------ 286.7/802.3 kB 1.3 MB/s eta 0:00:01
     -------------------------- ----------- 563.2/802.3 kB 2.2 MB/s eta 0:00:01
     --------------------------- ---------- 583.7/802.3 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 802.3/802.3 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyghelpers: filename=pyghelpers-1.1-py3-none-any.whl size=882583 sha256=e0044e5501c147fcc068d1a697751ff5fc7be43c1e89b0bf386a3eb4ccdb5ccb
  Stored in directory: c:\users\no

In [5]:
from typing import List
from pygame.locals import *
import pygwidgets
import pyghelpers
from Player import *
from Baddies import *
from Goodies import *

STATE_WAITING = 'waiting'
STATE_PLAYING = 'playing'
STATE_GAME_OVER = 'game over'

class ScenePlay(pyghelpers.Scene):
    def __init__(self, window):
        self.window = window

        self.controlsBackground = pygwidgets.Image(self.window, (0, GAME_HEIGHT), 'images/controlsBackground.jpg')
        self.quitButton = pygwidgets.CustomButton(self.window, (30, GAME_HEIGHT + 90), up='images/quitNormal.png', down='images/quitDown.png', over='images/quitOver.png', disabled='images/quitDisabled.png')
        self.highScoresButton = pygwidgets.CustomButton(self.window, (190, GAME_HEIGHT + 90), up='images/gotoHighScoresNormal.png', down='images/gotoHighScoresDown.png', over='images/gotoHighScoresOver.png', disabled='images/gotoHighScoresDisabled.png')
        self.newGameButton = pygwidgets.CustomButton(self.window, (450, GAME_HEIGHT + 90), up='images/startNewNormal.png', down='images/startNewDown.png', over='images/startNewOver.png', disabled='images/startNewDisabled.png', enterToActivate=True)
        self.soundCheckBox = pygwidgets.TextCheckBox(self.window, (430, GAME_HEIGHT + 17), 'Background music', True, textColor=WHITE)
        self.gameOverImage = pygwidgets.Image(self.window, (140, 180), 'images/gameOver.png')
        self.titleText = pygwidgets.DisplayText(self.window, (70, GAME_HEIGHT + 17), 'Score:                                 High Score:', fontSize=24, textColor=WHITE)
        self.scoreText = pygwidgets.DisplayText(self.window, (80, GAME_HEIGHT + 47), '0', fontSize=36, textColor=WHITE, justified='right')
        self.highScoreText = pygwidgets.DisplayText(self.window, (270, GAME_HEIGHT + 47), '', fontSize=36, textColor=WHITE, justified='right')

        pygame.mixer.music.load('sounds/background.mid')
        self.dingSound = pygame.mixer.Sound('sounds/ding.wav')
        self.gameOverSound = pygame.mixer.Sound('sounds/gameover.wav')

        # Instantiate objects
        self.oPlayer = Player(self.window)
        self.oBaddieMgr = BaddieMgr(self.window)
        self.oGoodieMgr = GoodieMgr(self.window)

        self.highestHighScore = 0
        self.lowestHighScore = 0
        self.backgroundMusic = True
        self.score = 0
        self.playingState = STATE_WAITING

    def getSceneKey(self) -> str:
        return SCENE_PLAY

    def enter(self, data: dict) -> None:
        self.getHiAndLowScores()

    def getHiAndLowScores(self) -> None:
        infoDict = self.request(SCENE_HIGH_SCORES, HIGH_SCORES_DATA)
        self.highestHighScore = infoDict['highest']
        self.highScoreText.setValue(self.highestHighScore)
        self.lowestHighScore = infoDict['lowest']

    def reset(self) -> None:
        self.score = 0
        self.scoreText.setValue(self.score)
        self.getHiAndLowScores()

        # Tell the managers to reset themselves
        self.oBaddieMgr.reset()
        self.oGoodieMgr.reset()

        if self.backgroundMusic:
            pygame.mixer.music.play(-1, 0.0)
        self.newGameButton.disable()
        self.highScoresButton.disable()
        self.soundCheckBox.disable()
        self.quitButton.disable()
        pygame.mouse.set_visible(False)

    def handleInputs(self, eventsList: List[pygame.event.Event], keyPressedList: List[int]) -> None:
        if self.playingState == STATE_PLAYING:
            self.handlePlayingEvents(eventsList)
        elif self.playingState == STATE_WAITING:
            self.handleWaitingEvents(eventsList)
        elif self.playingState == STATE_GAME_OVER:
            self.handleGameOverEvents(eventsList)

    def handlePlayingEvents(self, eventsList: List[pygame.event.Event]) -> None:
        for event in eventsList:
            if event.type == KEYDOWN and event.key == K_ESCAPE:
                self.quit()

            # Handle mouse-based events for the player (move player to mouse position)
            if event.type == MOUSEBUTTONDOWN:
                mouseX, mouseY = pygame.mouse.get_pos()
                playerRect = self.oPlayer.update(mouseX, mouseY)

            # Common button events (Pause, Restart, etc.)
            self.handleCommonButtonEvents(event)

    def handleWaitingEvents(self, eventsList: List[pygame.event.Event]) -> None:
        for event in eventsList:
            # Waiting state events: Start new game, view high scores, toggle sound, quit game
            if self.newGameButton.handleEvent(event):
                self.reset()
                self.playingState = STATE_PLAYING

            if self.highScoresButton.handleEvent(event):
                self.goToScene(SCENE_HIGH_SCORES)

            if self.soundCheckBox.handleEvent(event):
                self.backgroundMusic = self.soundCheckBox.getValue()

            if self.quitButton.handleEvent(event):
                self.quit()

    def handleGameOverEvents(self, eventsList: List[pygame.event.Event]) -> None:
        for event in eventsList:
            # Game Over events: Restart game, view high scores, quit game
            if self.newGameButton.handleEvent(event):
                self.reset()
                self.playingState = STATE_PLAYING

            if self.highScoresButton.handleEvent(event):
                self.goToScene(SCENE_HIGH_SCORES)

            if self.quitButton.handleEvent(event):
                self.quit()

    def handleCommonButtonEvents(self, event: pygame.event.Event) -> None:
        if self.quitButton.handleEvent(event):
            self.quit()

        if self.highScoresButton.handleEvent(event):
            self.goToScene(SCENE_HIGH_SCORES)

        if self.soundCheckBox.handleEvent(event):
            self.backgroundMusic = self.soundCheckBox.getValue()

    def update(self) -> None:
        if self.playingState != STATE_PLAYING:
            return  # only update when playing

        # Move the Player to the mouse position, get back its rect
        mouseX, mouseY = pygame.mouse.get_pos()
        playerRect = self.oPlayer.update(mouseX, mouseY)

        # Tell the GoodieMgr to move all Goodies
        nGoodiesHit = self.oGoodieMgr.update(playerRect)
        if nGoodiesHit > 0:
            self.dingSound.play()
            self.score = self.score + (nGoodiesHit * POINTS_FOR_GOODIE)

        # Tell the BaddieMgr to move all the Baddies
        nBaddiesEvaded = self.oBaddieMgr.update()
        self.score = self.score + (nBaddiesEvaded * POINTS_FOR_BADDIE_EVADED)
        
        self.scoreText.setValue(self.score)

        # Check if the Player has hit any Baddie
        if self.oBaddieMgr.hasPlayerHitBaddie(playerRect):
            pygame.mouse.set_visible(True)
            pygame.mixer.music.stop()

            self.gameOverSound.play()
            self.playingState = STATE_GAME_OVER
            self.draw()  # force drawing of game over message

            if self.score > self.lowestHighScore:
                scoreString = 'Your score: ' + str(self.score) + '\n'
                if self.score > self.highestHighScore:
                    dialogText = (scoreString +
                                   'is a new high score, CONGRATULATIONS!')
                else:
                    dialogText = (scoreString +
                                  'gets you on the high scores list.')

                result = showCustomYesNoDialog(self.window, dialogText)
                if result:  # navigate
                    self.goToScene(SCENE_HIGH_SCORES, self.score)  

            self.newGameButton.enable()
            self.highScoresButton.enable()
            self.soundCheckBox.enable()
            self.quitButton.enable()

    def draw(self) -> None:
        self.window.fill(BLACK)
    
        # Tell the managers to draw all the Baddies and Goodies
        self.oBaddieMgr.draw()
        self.oGoodieMgr.draw()
    
        # Tell the Player to draw itself
        self.oPlayer.draw()
    
        # Draw all the info at the bottom of the window
        self.controlsBackground.draw()
        self.titleText.draw()
        self.scoreText.draw()
        self.highScoreText.draw()
        self.soundCheckBox.draw()
        self.quitButton.draw()
        self.highScoresButton.draw()
        self.newGameButton.draw()

        if self.playingState == STATE_GAME_OVER:
            self.gameOverImage.draw()


In [1]:
#  Dodger main program
#
# Instantiates 3 scenes, creates and starts the scene manager
#
#  Original version by Al Sweigart from his book "Invent With Python"
#    (concept, graphics, and sounds used by permission from Al Sweigart)

# 1 - Import packages
import os

try:
    # Try to use __file__ to get the script directory
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # If __file__ is not defined (e.g., in a notebook or certain IDEs)
    script_dir = os.getcwd()  # Fall back to the current working directory

# Change to the script's directory
os.chdir(script_dir)


import pygame
import pyghelpers
from SceneSplash import *
from ScenePlay import *
from SceneHighScores import *

# 2 - Define constants
FRAMES_PER_SECOND = 40

# 3 - Initialize the world
pygame.init()
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))

# 4 - Load assets: image(s), sounds,  etc.

# 5 - Initialize variables
# Instantiate all scenes and store them in a list
scenesList = [SceneSplash(window),
                    SceneHighScores(window),
                    ScenePlay(window)]

# Create the scene manager, passing in the scenes list and the FPS
oSceneMgr = pyghelpers.SceneMgr(scenesList, FRAMES_PER_SECOND)

# Tell the Scene Manager to start running
oSceneMgr.run()


pygame-ce 2.5.3 (SDL 2.30.12, Python 3.11.7)


SystemExit: 

C:\Users\Notab\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
